## 10. Collaboration

### 88 Know How to Break Circular Dependencies

```python
# recursive_import_bad/dialog.py

import app

class Dialog:
    def __init__(self, save_dir):
        self.save_dir = save_dir

save_dialog = Dialog(app.prefs.get('save_dir'))

def show():
    print('Showing the dialog!')
```

```python
# recursive_import_bad/app.py

import dialog

class Prefs:
    def get(self, name):
        pass

prefs = Prefs()
dialog.show()
```

```python
# recursive_import_bad/main.py

import app
```

```shell
$ python main.py 
Traceback (most recent call last):
  File "main.py", line 17, in <module>
    import app
  File "/home/jay/_doc/spike/python/effective/88/recursive_import_bad/app.py", line 17, in <module>
    import dialog
  File "/home/jay/_doc/spike/python/effective/88/recursive_import_bad/dialog.py", line 23, in <module>
    save_dialog = Dialog(app.prefs.get('save_dir'))
AttributeError: partially initialized module 'app' has no attribute 'prefs' (most likely due to a circular import)

```

모듈이 임포트되면 파이썬이 하는 일을 깊이 우선순위(depth first order)로 나타내면

1. `sys.path`에서 모듈 위치를 검색한다.
1. 모듈의 코드를 로딩하고 컴파일되는지 확인한다.
1. 임포트할 모듈에 상응하는 빈 모듈 객체를 만든다.
1. 모듈을 `sys.modules`에 넣는다.
1. 모듈 객체에 있는 코드를 실행해서 모듈의 내용을 정의한다.

순환 의존 관계에서 문제는 어떤 모듈의 애트리뷰트를 정의하는 코드(5단계)가 실제로 실행되기 전까지는 모듈 애트리뷰트가 정의되지 않는다는 점이다. 하지만 모듈 자체는 `import` 문을 사용해서 `sys.modules`에 추가되지마자(4단계) 로드할 수 있다.

`app` 모듈은 `dialog` 모듈을 임포트하고, `dialog` 모듈은 `app` 모듈을 임포트해서 아직 실행되지 않은 `app` 모듈의 `prefs` 애트리뷰트에 접근하고, 그래서 `AttributeError` 에러가 발생하고...

`prefs`를 별도의 유틸리티 모듈로 옮기고 `app`과 `dialog` 모두 그 유틸리티 모듈을 임포트해서 순환 임포트를 피할 수 있다. 하지만 리팩터링이 너무 어렵거나 이런 식의 명확한 구분이 불가능한 경우도 있다.

#### 임포트 순서 바꾸기

```python
# recursive_import_ordering/app.py

class Prefs:
    def get(self, name):
        pass

prefs = Prefs()

import dialog  # Moved
dialog.show()
```

```shell
$ python main.py 
Showing the dialog!
```

이렇게 하지 말자!

#### 임포트, 설정, 실행

```python
# recursive_import_nosideeffects/dialog.py

import app

class Dialog:
    def __init__(self):
        pass

save_dialog = Dialog()

def show():
    print('Showing the dialog!')

def configure():
    save_dialog.save_dir = app.prefs.get('save_dir')
```

```python
# recursive_import_nosideeffects/app.py

import dialog

class Prefs:
    def get(self, name):
        pass

prefs = Prefs()

def configure():
    pass
```

```python
# recursive_import_nosideeffects/main.py

import app
import dialog

app.configure()
dialog.configure()

dialog.show()
```

```shell
$ python main.py 
Showing the dialog!
```

#### 동적 임포트

```python
# recursive_import_dynamic/dialog.py

class Dialog:
    def __init__(self):
        pass

save_dialog = Dialog()

def show():
    import app  # Dynamic import
    save_dialog.save_dir = app.prefs.get('save_dir')
    print('Showing the dialog!')
```

```python
# recursive_import_dynamic/app.py

import dialog

class Prefs:
    def get(self, name):
        pass

prefs = Prefs()
dialog.show()
```

```python
# recursive_import_dynamic/main.py

import app
```

```shell
$ python main.py 
Showing the dialog!
```

`import` 문의 비용이 무시하지 못할 만큼 크며, ... 동적 임포트를 사용하면 임포트 실행을 미루기 때문에 실행 시점에 예기치 못한 오류로 인해 놀랄 수도 있다. ... 이런 단점을 감수하는 것이 프로그램 전체 구조를 바꾸는 것보다 더 나은 경우...

> - 두 모듈이 임포트 시점에 서로를 호출하면 순환 의존 관계가 생긴다. 순환 의존 관계가 있으면 프로그램이 시작되다가 오류가 발생하면서 중단될 수 있다.
> - 순환 의존 관계를 깨는 가장 좋은 방법은 상호 의존 관계를 의존 관계 트리의 맨 아래에 위치한 별도의 모듈로 리팩토링하는 것이다.
> - 동적 임포트는 리팩터링과 복잡도 증가를 최소화하면서 모듈 간의 의존 관계를 깨는 가장 단순한 방법이다.